In [66]:
import pandas as pd

In [ ]:
import pandas as pdimport pandas as pd

In [67]:
df = pd.read_csv('train.csv')
df

## preprocessing and feature eng.

In [68]:
#in this code i did ordinal encoding on categorical variables (cut,color,clarity) to give each value in the column its position
c = {"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
df["cut_ordinal"]= df['cut'].map(c)
b = {'D' : 7 , 'E':6, 'F':5, 'G' : 4, 'H':3, 'I':2, 'J':1}
df['color_ordinal'] = df['color'].map(b)
e = {'I1':1, 'SI2':2, 'SI1':2, 'VS2':3, 'VS1':4, 'VVS2':5, 'VVS1':6, 'IF':7 }
df['clarity_ordinal'] = df['clarity'].map(e)

In [69]:
# added some features to make the model training better
df['volume'] = df['x'] * df['y'] * df['z']
df['density'] = df['carat'] / df['volume']
df['surface_area'] = 2 * (df['x'] * df['y'] + df['x'] * df['z'] + df['y'] * df['z'])
df['depth_percentage'] = (df['depth'] / ((df['x'] + df['y'] + df['z']) / 3)) * 100
df['length_ratio_xy'] = df['x'] / df['y']
df['length_ratio_xz'] = df['x'] / df['z']
df['length_ratio_yz'] = df['y'] / df['z']

In [70]:
df = df.drop(['Id'],axis = 1)

In [71]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 23 mins
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_4t4_3ss19e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,73.1 Mb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [72]:
h2o_data = h2o.H2OFrame(df)
h2o_data

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


carat,cut,color,clarity,depth,table,price,x,y,z,cut_ordinal,color_ordinal,clarity_ordinal,volume,density,surface_area,depth_percentage,length_ratio_xy,length_ratio_xz,length_ratio_yz
1.06,Ideal,I,SI2,61.8,57,4270,6.57,6.6,4.07,5,2,2,176.483,0.00600623,193.928,1075.41,0.995455,1.61425,1.62162
1.51,Premium,G,VVS2,60.9,58,15164,7.38,7.42,4.51,4,4,5,246.966,0.00611421,243.015,946.142,0.994609,1.63636,1.64523
0.32,Ideal,F,VS2,61.3,56,828,4.43,4.41,2.71,5,5,3,52.9434,0.00604419,86.9854,1592.21,1.00454,1.63469,1.62731
0.53,Ideal,G,VS2,61.2,56,1577,5.19,5.22,3.19,5,4,3,86.4228,0.00613264,120.599,1350,0.994253,1.62696,1.63636
0.7,Premium,H,VVS2,61,57,2596,5.76,5.72,3.5,4,3,5,115.315,0.00607032,146.254,1221.63,1.00699,1.64571,1.63429
0.9,Very Good,G,IF,63.1,58,5670,6.18,6.09,3.87,3,4,7,145.652,0.00617911,170.242,1172.86,1.01478,1.5969,1.57364
0.71,Very Good,F,VS1,61.8,60,3017,5.66,5.7,3.51,3,5,4,113.24,0.00626989,144.271,1246.81,0.992982,1.61254,1.62393
0.7,Ideal,F,SI1,61.3,57,2288,5.72,5.77,3.52,5,5,2,116.175,0.00602537,146.898,1225.18,0.991334,1.625,1.6392
0.59,Ideal,D,VS1,60.9,56,2175,5.39,5.44,3.3,5,7,4,96.7613,0.00609748,130.121,1292.99,0.990809,1.63333,1.64848
0.31,Very Good,G,VVS1,59.8,62,761,4.35,4.41,2.62,3,4,6,50.2608,0.00616783,84.2694,1576.45,0.986395,1.66031,1.68321


In [74]:
import h2o
# i will transform those column into categorical factors

h2o_data['clarity'] = h2o_data['clarity'].asfactor()
h2o_data['color'] = h2o_data['color'].asfactor()
h2o_data['cut'] = h2o_data['cut'].asfactor()



In [75]:
from h2o.automl import H2OAutoML
h2o.init()

aml = H2OAutoML(max_runtime_secs=700,seed = 789)

aml.train(y="price", training_frame=h2o_data)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 25 mins
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_4t4_3ss19e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,70.0 Mb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


AutoML progress: |█
16:12:25.200: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/6
# GBM base models (used / total),4/4
# DRF base models (used / total),0/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [76]:
h2o.init()

best_model = aml.leader
print(best_model)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 39 mins
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_4t4_3ss19e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,134.2 Mb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_10_20240515_161225


Model Summary for Stacked Ensemble: 
key                                   value
------------------------------------  ----------------
Stacking strategy                     cross_validation
Number of base models (used / total)  4/6
# GBM base models (used / total)      4/4
# DRF base models (used / total)      0/1
# GLM base models (used / total)      0/1
Metalearner algorithm                 GLM
Metalearner fold assignment scheme    Random
Metalearner nfolds                    5
Metalearner fold_column
Custom metalearner hyperparameters    None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 145792.1896843806
RMSE: 381.8274344312894
MAE: 200.93321233644622
RMSLE: 0.0705795592034377
Mean Residual Deviance: 145792.1896843806
R^2: 0.9905637666312739
Null degrees of freedom: 9984
Residual degrees of freedom: 9980
Null deviance: 15

## test data

In [90]:
test_data = pd.read_csv('test (1).csv')
test_data

,Id,carat,cut,color,clarity,depth,table,x,y,z
0,1,0.34,Ideal,G,VVS2,61.1,57.0,4.52,4.48,2.75
1,2,0.71,Premium,E,VS2,62.7,58.0,5.74,5.68,3.58
2,3,0.44,Very Good,I,VS1,62.8,56.0,4.83,4.88,3.05
3,4,0.81,Premium,E,SI2,60.1,59.0,6.09,6.03,3.65
4,5,0.40,Ideal,G,VVS1,61.2,56.0,4.74,4.80,2.92
...,...,...,...,...,...,...,...,...,...,...
10783,10784,0.57,Ideal,H,VS1,60.9,56.0,5.34,5.36,3.26
10784,10785,1.05,Ideal,G,VS2,60.8,57.0,6.65,6.58,4.02
10785,10786,0.71,Ideal,E,VVS1,62.3,55.0,5.68,5.72,3.55
10786,10787,1.11,Premium,E,SI2,61.0,60.0,6.68,6.66,4.07


In [91]:
j = {"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
test_data["cut_ordinal"]= test_data['cut'].map(c)
k = {'D' : 7 , 'E':6, 'F':5, 'G' : 4, 'H':3, 'I':2, 'J':1}
test_data['color_ordinal'] = test_data['color'].map(b)
l = {'I1':1, 'SI2':2, 'SI1':2, 'VS2':3, 'VS1':4, 'VVS2':5, 'VVS1':6, 'IF':7 }
test_data['clarity_ordinal'] = test_data['clarity'].map(e)


In [92]:
test_data['volume'] = test_data['x'] * test_data['y'] * test_data['z']
test_data['density'] = test_data['carat'] / test_data['volume']
test_data['surface_area'] = 2 * (test_data['x'] * test_data['y'] + test_data['x'] * test_data['z'] + test_data['y'] * test_data['z'])
test_data['depth_percentage'] = (test_data['depth'] / ((test_data['x'] + test_data['y'] + test_data['z']) / 3)) * 100
test_data['length_ratio_xy'] = test_data['x'] / test_data['y']
test_data['length_ratio_xz'] = test_data['x'] / test_data['z']
test_data['length_ratio_yz'] = test_data['y'] / test_data['z']

In [94]:
test_data

,Id,carat,cut,color,clarity,depth,table,x,y,z,cut_ordinal,color_ordinal,clarity_ordinal,volume,density,surface_area,depth_percentage,length_ratio_xy,length_ratio_xz,length_ratio_yz
0,1,0.34,Ideal,G,VVS2,61.1,57.0,4.52,4.48,2.75,5,4,5,55.686400,0.006106,89.9992,1560.000000,1.008929,1.643636,1.629091
1,2,0.71,Premium,E,VS2,62.7,58.0,5.74,5.68,3.58,4,6,3,116.719456,0.006083,146.9736,1254.000000,1.010563,1.603352,1.586592
2,3,0.44,Very Good,I,VS1,62.8,56.0,4.83,4.88,3.05,3,2,4,71.889720,0.006120,106.3718,1476.489028,0.989754,1.583607,1.600000
3,4,0.81,Premium,E,SI2,60.1,59.0,6.09,6.03,3.65,4,6,2,134.037855,0.006043,161.9214,1143.310082,1.009950,1.668493,1.652055
4,5,0.40,Ideal,G,VVS1,61.2,56.0,4.74,4.80,2.92,5,4,6,66.435840,0.006021,101.2176,1473.515249,0.987500,1.623288,1.643836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10783,10784,0.57,Ideal,H,VS1,60.9,56.0,5.34,5.36,3.26,5,3,4,93.309024,0.006109,127.0088,1308.739255,0.996269,1.638037,1.644172
10784,10785,1.05,Ideal,G,VS2,60.8,57.0,6.65,6.58,4.02,5,4,3,175.903140,0.005969,193.8832,1057.391304,1.010638,1.654229,1.636816
10785,10786,0.71,Ideal,E,VVS1,62.3,55.0,5.68,5.72,3.55,5,6,6,115.338080,0.006156,145.9192,1250.167224,0.993007,1.600000,1.611268
10786,10787,1.11,Premium,E,SI2,61.0,60.0,6.68,6.66,4.07,4,6,2,181.069416,0.006130,197.5652,1051.120046,1.003003,1.641278,1.636364


In [93]:
h2o_train = h2o.H2OFrame(test_data)
h2o_train

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Id,carat,cut,color,clarity,depth,table,x,y,z,cut_ordinal,color_ordinal,clarity_ordinal,volume,density,surface_area,depth_percentage,length_ratio_xy,length_ratio_xz,length_ratio_yz
1,0.34,Ideal,G,VVS2,61.1,57,4.52,4.48,2.75,5,4,5,55.6864,0.00610562,89.9992,1560,1.00893,1.64364,1.62909
2,0.71,Premium,E,VS2,62.7,58,5.74,5.68,3.58,4,6,3,116.719,0.00608296,146.974,1254,1.01056,1.60335,1.58659
3,0.44,Very Good,I,VS1,62.8,56,4.83,4.88,3.05,3,2,4,71.8897,0.00612049,106.372,1476.49,0.989754,1.58361,1.6
4,0.81,Premium,E,SI2,60.1,59,6.09,6.03,3.65,4,6,2,134.038,0.00604307,161.921,1143.31,1.00995,1.66849,1.65205
5,0.4,Ideal,G,VVS1,61.2,56,4.74,4.8,2.92,5,4,6,66.4358,0.00602085,101.218,1473.52,0.9875,1.62329,1.64384
6,1.27,Ideal,F,VS1,61.6,55,6.97,7.03,4.31,5,5,4,211.186,0.00601365,218.678,1009.28,0.991465,1.61717,1.63109
7,1.01,Very Good,G,VS2,61.5,56,6.41,6.47,3.96,3,4,3,164.232,0.00614984,184.955,1095.61,0.990726,1.61869,1.63384
8,1.24,Fair,F,SI2,64.9,60,6.7,6.63,4.32,1,5,2,191.899,0.00646174,204.013,1103.12,1.01056,1.55093,1.53472
9,0.71,Fair,I,VS2,61.6,67,5.76,5.61,3.5,1,2,3,113.098,0.00627776,144.217,1242.77,1.02674,1.64571,1.60286
10,0.37,Very Good,G,VS1,61.5,58,4.57,4.6,2.82,3,4,4,59.282,0.00624135,93.7628,1538.78,0.993478,1.62057,1.63121


In [95]:
h2o_train['clarity'] = h2o_train['clarity'].asfactor()
h2o_train['color'] = h2o_train['color'].asfactor()
h2o_train['cut'] = h2o_train['cut'].asfactor()

In [99]:
h2o_train

Id,carat,cut,color,clarity,depth,table,x,y,z,cut_ordinal,color_ordinal,clarity_ordinal,volume,density,surface_area,depth_percentage,length_ratio_xy,length_ratio_xz,length_ratio_yz
1,0.34,Ideal,G,VVS2,61.1,57,4.52,4.48,2.75,5,4,5,55.6864,0.00610562,89.9992,1560,1.00893,1.64364,1.62909
2,0.71,Premium,E,VS2,62.7,58,5.74,5.68,3.58,4,6,3,116.719,0.00608296,146.974,1254,1.01056,1.60335,1.58659
3,0.44,Very Good,I,VS1,62.8,56,4.83,4.88,3.05,3,2,4,71.8897,0.00612049,106.372,1476.49,0.989754,1.58361,1.6
4,0.81,Premium,E,SI2,60.1,59,6.09,6.03,3.65,4,6,2,134.038,0.00604307,161.921,1143.31,1.00995,1.66849,1.65205
5,0.4,Ideal,G,VVS1,61.2,56,4.74,4.8,2.92,5,4,6,66.4358,0.00602085,101.218,1473.52,0.9875,1.62329,1.64384
6,1.27,Ideal,F,VS1,61.6,55,6.97,7.03,4.31,5,5,4,211.186,0.00601365,218.678,1009.28,0.991465,1.61717,1.63109
7,1.01,Very Good,G,VS2,61.5,56,6.41,6.47,3.96,3,4,3,164.232,0.00614984,184.955,1095.61,0.990726,1.61869,1.63384
8,1.24,Fair,F,SI2,64.9,60,6.7,6.63,4.32,1,5,2,191.899,0.00646174,204.013,1103.12,1.01056,1.55093,1.53472
9,0.71,Fair,I,VS2,61.6,67,5.76,5.61,3.5,1,2,3,113.098,0.00627776,144.217,1242.77,1.02674,1.64571,1.60286
10,0.37,Very Good,G,VS1,61.5,58,4.57,4.6,2.82,3,4,4,59.282,0.00624135,93.7628,1538.78,0.993478,1.62057,1.63121


In [101]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

h2o.init()

predictions = best_model.predict(h2o_train)

predictions_df = predictions.as_data_frame()

predictions_df['Id'] = data['Id']

predictions_df = predictions_df[['Id','predict']]

predictions_df = predictions_df.rename(columns={'predict': 'price'})
predictions_df = predictions_df.rename(columns={'Id': 'Id'})

print(predictions_df)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 56 mins
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_4t4_3ss19e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,328.9 Mb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


C:\Users\4t4\anaconda3\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


          Id         price
0          1    964.381049
1          2   2812.757053
2          3    846.905488
3          4   2778.044761
4          5   1090.217293
...      ...           ...
10783  10784   1744.392486
10784  10785   6522.807362
10785  10786   4539.560259
10786  10787   4740.305720
10787  10788  13832.815192

[10788 rows x 2 columns]


In [102]:
#predictions_df.to_csv('test100fk0.csv', index=False)
predictions_df.to_csv('h2o_final.csv', index=False)
